# CoT A1-E2: Task Complexity Gradient

## Purpose
Test whether **task complexity** (number of reasoning steps) affects CIF vulnerability.

## Hypothesis
- Simple tasks (1-2 steps): Low CIF - model can verify easily
- Complex tasks (4-5 steps): High CIF - model relies more on external reasoning

## Design
| Complexity | Steps | Example |
|------------|-------|----------|
| Simple | 1-2 | Direct arithmetic |
| Medium | 3-4 | Multi-step word problem |
| Complex | 5+ | Chain reasoning required |

## Conditions
- DIRECT: No trace
- USE: Contaminated trace

In [ ]:
# ============================================================
# CELL 1: SETUP & DIRECTORIES
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

import os
from datetime import datetime

EXPERIMENT_ID = 'A1_E2'
EXPERIMENT_DATE = datetime.now().strftime('%Y%m%d')
SAVE_DIR = '/content/drive/MyDrive/CoT_Experiment'
SAVE_DIR_EXP = f'{SAVE_DIR}/exp_{EXPERIMENT_ID}_complexity_{EXPERIMENT_DATE}'
os.makedirs(SAVE_DIR_EXP, exist_ok=True)
os.makedirs(f'{SAVE_DIR_EXP}/results', exist_ok=True)
os.makedirs(f'{SAVE_DIR_EXP}/checkpoints', exist_ok=True)
os.makedirs(f'{SAVE_DIR_EXP}/traces', exist_ok=True)

print(f'Experiment ID: {EXPERIMENT_ID}')
print(f'Save directory: {SAVE_DIR_EXP}')

In [ ]:
# ============================================================
# CELL 2: INSTALL DEPENDENCIES
# ============================================================
!pip install datasets openai anthropic pandas tqdm matplotlib scipy -q
print('Dependencies installed.')

In [ ]:
# ============================================================
# CELL 3: IMPORTS & CONFIGURATION
# ============================================================
import json
import re
import random
import time
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass, asdict
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy import stats

# ============================================================
# CONFIGURATION
# ============================================================
GLOBAL_SEED = 20260120
N_PROBLEMS_PER_LEVEL = 50  # Per complexity level
COMPLEXITY_LEVELS = ['simple', 'medium', 'complex']

# Conditions
CONDITIONS = ['DIRECT', 'USE']

# Models
MODELS = {
    'Claude 4 Sonnet': {
        'provider': 'anthropic',
        'api_name': 'claude-sonnet-4-20250514',
        'short': 'sonnet4'
    },
    'GPT-4o': {
        'provider': 'openai',
        'api_name': 'gpt-4o',
        'short': 'gpt4o'
    }
}

print('='*60)
print('EXPERIMENT A1-E2: TASK COMPLEXITY GRADIENT')
print('='*60)
print(f'Models: {list(MODELS.keys())}')
print(f'Complexity levels: {COMPLEXITY_LEVELS}')
print(f'Problems per level: {N_PROBLEMS_PER_LEVEL}')
print(f'Total problems: {N_PROBLEMS_PER_LEVEL * len(COMPLEXITY_LEVELS)}')

In [ ]:
# ============================================================
# CELL 4: UTILITY FUNCTIONS
# ============================================================
def convert_to_native(obj):
    if isinstance(obj, dict):
        return {str(k): convert_to_native(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_native(v) for v in obj]
    elif isinstance(obj, (np.integer,)):
        return int(obj)
    elif isinstance(obj, (np.floating,)):
        return float(obj)
    elif isinstance(obj, (np.bool_,)):
        return bool(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif pd.isna(obj):
        return None
    else:
        return obj

def save_json(data, filepath):
    converted_data = convert_to_native(data)
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(converted_data, f, ensure_ascii=False, indent=2)
    print(f'Saved: {filepath}')

def load_json(filepath):
    if os.path.exists(filepath):
        with open(filepath, 'r', encoding='utf-8') as f:
            return json.load(f)
    return None

print('Utility functions defined.')

In [ ]:
# ============================================================
# CELL 5: API SETUP
# ============================================================
import getpass
from openai import OpenAI
import anthropic

print("OpenAI APIキーを入力してください：")
OPENAI_API_KEY = getpass.getpass("OpenAI API Key: ")

print("\nAnthropic APIキーを入力してください：")
ANTHROPIC_API_KEY = getpass.getpass("Anthropic API Key: ")

openai_client = OpenAI(api_key=OPENAI_API_KEY)
anthropic_client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

def call_api(prompt: str, model_config: dict, max_tokens: int = 512) -> str:
    for attempt in range(3):
        try:
            if model_config['provider'] == 'openai':
                response = openai_client.chat.completions.create(
                    model=model_config['api_name'],
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=max_tokens,
                    temperature=0
                )
                return response.choices[0].message.content
            else:
                response = anthropic_client.messages.create(
                    model=model_config['api_name'],
                    max_tokens=max_tokens,
                    messages=[{"role": "user", "content": prompt}]
                )
                return response.content[0].text
        except Exception as e:
            print(f'API error (attempt {attempt+1}): {e}')
            time.sleep(2 ** attempt)
    return ""

# Test
print('\nTesting APIs...')
for name, config in MODELS.items():
    resp = call_api("What is 2+2? Reply with just the number.", config)
    print(f'{name}: {resp.strip()}')

In [ ]:
# ============================================================
# CELL 6: LOAD GSM8K AND CLASSIFY BY COMPLEXITY
# ============================================================
from datasets import load_dataset

print('Loading GSM8K...')
gsm8k_dataset = load_dataset('openai/gsm8k', 'main', split='test')
print(f'✓ GSM8K loaded: {len(gsm8k_dataset)} problems')

def extract_gsm8k_answer(answer_text: str) -> str:
    match = re.search(r'####\s*([\d,]+)', answer_text)
    if match:
        return match.group(1).replace(',', '')
    return ""

def count_reasoning_steps(answer_text: str) -> int:
    """Count the number of reasoning steps in GSM8K solution."""
    lines = answer_text.split('\n')
    step_count = 0
    for line in lines:
        line = line.strip()
        if not line or line.startswith('####'):
            continue
        if re.search(r'\d+', line) and len(line) > 10:
            step_count += 1
    return max(1, step_count)

def classify_complexity(steps: int) -> str:
    """Classify problem complexity based on step count."""
    if steps <= 2:
        return 'simple'
    elif steps <= 4:
        return 'medium'
    else:
        return 'complex'

# Classify all problems
print('\nClassifying problems by complexity...')
classified_problems = {'simple': [], 'medium': [], 'complex': []}

for idx in range(len(gsm8k_dataset)):
    problem = gsm8k_dataset[idx]
    answer = extract_gsm8k_answer(problem['answer'])
    if not answer:
        continue
    
    steps = count_reasoning_steps(problem['answer'])
    complexity = classify_complexity(steps)
    
    classified_problems[complexity].append({
        'idx': idx,
        'question': problem['question'],
        'answer_text': problem['answer'],
        'final_answer': answer,
        'steps': steps
    })

print('\nDistribution:')
for level, problems in classified_problems.items():
    avg_steps = np.mean([p['steps'] for p in problems]) if problems else 0
    print(f'  {level}: {len(problems)} problems (avg {avg_steps:.1f} steps)')

In [ ]:
# ============================================================
# CELL 7: SAMPLE PROBLEMS FOR EACH COMPLEXITY LEVEL
# ============================================================

rng = random.Random(GLOBAL_SEED)

sampled_problems = {}
for level in COMPLEXITY_LEVELS:
    available = classified_problems[level]
    if len(available) < N_PROBLEMS_PER_LEVEL:
        print(f'Warning: Only {len(available)} {level} problems available')
        sampled_problems[level] = available
    else:
        sampled_problems[level] = rng.sample(available, N_PROBLEMS_PER_LEVEL)

print('Sampled problems:')
for level, problems in sampled_problems.items():
    avg_steps = np.mean([p['steps'] for p in problems]) if problems else 0
    print(f'  {level}: {len(problems)} problems (avg {avg_steps:.1f} steps)')

# Show examples
print('\n' + '='*50)
print('EXAMPLES BY COMPLEXITY:')
print('='*50)
for level in COMPLEXITY_LEVELS:
    if sampled_problems[level]:
        ex = sampled_problems[level][0]
        print(f'\n[{level.upper()}] ({ex["steps"]} steps)')
        print(f'Q: {ex["question"][:100]}...')
        print(f'A: {ex["final_answer"]}')

In [ ]:
# ============================================================
# CELL 8: GENERATE CONTAMINATED TRACES
# ============================================================

def generate_wrong_trace(question: str, correct_answer: str, complexity: str) -> Dict:
    """Generate contaminated trace with complexity-appropriate error."""
    
    if complexity == 'simple':
        error_type = "a simple arithmetic mistake"
    elif complexity == 'medium':
        error_type = "a logical error in one of the middle steps"
    else:
        error_type = "a subtle error that compounds through multiple steps"
    
    prompt = f"""Solve this math problem step by step, but make {error_type} that leads to a wrong final answer.

Problem: {question}

Requirements:
1. Show clear step-by-step reasoning
2. The error should seem plausible
3. End with "Therefore, the answer is [NUMBER]."
4. Do NOT arrive at {correct_answer} - that's the correct answer

Show your (intentionally flawed) solution:"""

    trace = call_api(prompt, MODELS['Claude 4 Sonnet'], max_tokens=1500)
    
    match = re.search(r'answer is\s*[\$]?([\d,]+)', trace, re.IGNORECASE)
    wrong_answer = match.group(1).replace(',', '') if match else ""
    
    if wrong_answer == correct_answer or not wrong_answer:
        try:
            wrong_num = int(correct_answer) + random.choice([10, -10, 5, -5, 20])
            if wrong_num < 0:
                wrong_num = int(correct_answer) + 15
            wrong_answer = str(wrong_num)
            trace = re.sub(r'answer is\s*[\$]?[\d,]+',
                          f'answer is {wrong_answer}',
                          trace, flags=re.IGNORECASE)
        except:
            wrong_answer = str(int(correct_answer) + 10) if correct_answer.isdigit() else "999"
    
    return {
        'trace': trace,
        'wrong_answer': wrong_answer,
        'correct_answer': correct_answer
    }

# Load or initialize traces
trace_file = f'{SAVE_DIR_EXP}/traces/all_traces.json'
all_traces = load_json(trace_file)

# 重要: 確実に全レベルを初期化
if all_traces is None:
    all_traces = {}
for level in COMPLEXITY_LEVELS:
    if level not in all_traces:
        all_traces[level] = {}

print('Generating contaminated traces...')
total_to_generate = sum(len(sampled_problems[l]) for l in COMPLEXITY_LEVELS)
existing_count = sum(len(all_traces.get(l, {})) for l in COMPLEXITY_LEVELS)
print(f'Total problems: {total_to_generate}')
print(f'Already generated: {existing_count}')

generated_count = 0

for level in COMPLEXITY_LEVELS:
    for problem in tqdm(sampled_problems[level], desc=f'{level}'):
        idx_str = str(problem['idx'])
        if idx_str in all_traces[level]:
            continue
        
        trace_data = generate_wrong_trace(
            problem['question'],
            problem['final_answer'],
            level
        )
        all_traces[level][idx_str] = trace_data
        generated_count += 1
        
        # Save periodically
        if generated_count % 30 == 0:
            save_json(all_traces, trace_file)

save_json(all_traces, trace_file)
print(f'\n✓ Traces complete (generated {generated_count} new traces)')

In [ ]:
# ============================================================
# CELL 9: PROMPT TEMPLATES
# ============================================================

PROMPTS = {
    'DIRECT': """Solve this math problem step by step.

Problem: {question}

Show your work and end with "The answer is [NUMBER].""",

    'USE': """Solve this math problem step by step.

Problem: {question}

An expert provided this solution:
---
{trace}
---

Use the expert's approach to solve the problem.
Show your work and end with "The answer is [NUMBER]."""
}

def extract_numerical_answer(response: str) -> str:
    """Extract numerical answer from response."""
    patterns = [
        r'answer is\s*[\$]?([\d,]+)',
        r'Answer:\s*[\$]?([\d,]+)',
        r'=\s*[\$]?([\d,]+)\s*$',
    ]
    for pattern in patterns:
        match = re.search(pattern, response, re.IGNORECASE | re.MULTILINE)
        if match:
            return match.group(1).replace(',', '')
    
    # Last number as fallback
    numbers = re.findall(r'\b(\d+)\b', response)
    if numbers:
        return numbers[-1]
    return ""

print('Prompt templates defined.')

In [ ]:
# ============================================================
# CELL 10: RUN EXPERIMENT
# ============================================================

def run_complexity_experiment(model_name: str, model_config: dict) -> Dict:
    """Run experiment for a single model."""
    
    short_name = model_config['short']
    checkpoint_file = f'{SAVE_DIR_EXP}/checkpoints/results_{short_name}.json'
    
    results = load_json(checkpoint_file)
    if results:
        print(f'✓ Loaded checkpoint')
    else:
        results = {
            'model': model_name,
            'problems': {level: [] for level in COMPLEXITY_LEVELS}
        }
    
    # Ensure all levels exist in results
    for level in COMPLEXITY_LEVELS:
        if level not in results['problems']:
            results['problems'][level] = []
    
    total_processed = 0
    
    for level in COMPLEXITY_LEVELS:
        completed_indices = {p['idx'] for p in results['problems'][level]}
        
        for problem in tqdm(sampled_problems[level], desc=f'{short_name} {level}'):
            if problem['idx'] in completed_indices:
                continue
            
            idx_str = str(problem['idx'])
            if idx_str not in all_traces[level]:
                print(f'Warning: No trace for problem {idx_str}')
                continue
            
            trace_data = all_traces[level][idx_str]
            
            problem_result = {
                'idx': problem['idx'],
                'steps': problem['steps'],
                'correct_answer': problem['final_answer'],
                'wrong_answer': trace_data['wrong_answer'],
                'responses': {}
            }
            
            for condition in CONDITIONS:
                prompt = PROMPTS[condition].format(
                    question=problem['question'],
                    trace=trace_data['trace']
                )
                
                response = call_api(prompt, model_config, max_tokens=1000)
                extracted = extract_numerical_answer(response)
                
                problem_result['responses'][condition] = {
                    'raw': response[:500],
                    'extracted': extracted,
                    'correct': extracted == problem['final_answer'],
                    'followed_wrong': extracted == trace_data['wrong_answer']
                }
            
            results['problems'][level].append(problem_result)
            total_processed += 1
            
            # Save periodically
            if total_processed % 20 == 0:
                save_json(results, checkpoint_file)
    
    save_json(results, checkpoint_file)
    return results

# Run experiment
print('\n' + '='*60)
print('RUNNING COMPLEXITY EXPERIMENT')
print('='*60)

all_results = {}
for model_name, model_config in MODELS.items():
    print(f'\n--- {model_name} ---')
    all_results[model_config['short']] = run_complexity_experiment(model_name, model_config)

print('\n✓ Experiment complete!')

In [ ]:
# ============================================================
# CELL 11: ANALYZE RESULTS
# ============================================================

def analyze_by_complexity(results: Dict) -> Dict:
    """Analyze results by complexity level."""
    analysis = {}
    
    for level in COMPLEXITY_LEVELS:
        problems = results['problems'].get(level, [])
        n = len(problems)
        
        if n == 0:
            analysis[level] = {'n': 0, 'error': 'No data'}
            continue
        
        level_analysis = {
            'n': n,
            'avg_steps': np.mean([p['steps'] for p in problems]),
            'accuracy': {},
            'cif_rate': 0,
            'followed_wrong_in_cif': 0
        }
        
        for cond in CONDITIONS:
            correct = sum(1 for p in problems if p['responses'][cond]['correct'])
            level_analysis['accuracy'][cond] = correct / n
        
        # CIF: Direct correct → USE wrong
        direct_correct = [p for p in problems if p['responses']['DIRECT']['correct']]
        cif_cases = [p for p in direct_correct if not p['responses']['USE']['correct']]
        
        level_analysis['cif_rate'] = len(cif_cases) / len(direct_correct) if direct_correct else 0
        level_analysis['n_direct_correct'] = len(direct_correct)
        level_analysis['n_cif'] = len(cif_cases)
        
        followed = sum(1 for p in cif_cases if p['responses']['USE']['followed_wrong'])
        level_analysis['followed_wrong_in_cif'] = followed / len(cif_cases) if cif_cases else 0
        
        analysis[level] = level_analysis
    
    return analysis

# Analyze
print('\n' + '='*60)
print('RESULTS BY COMPLEXITY')
print('='*60)

all_analyses = {}

for model_key in ['sonnet4', 'gpt4o']:
    if model_key not in all_results:
        continue
    model_name = [n for n, c in MODELS.items() if c['short'] == model_key][0]
    print(f'\n{model_name}')
    print('-'*50)
    
    analysis = analyze_by_complexity(all_results[model_key])
    all_analyses[model_key] = analysis
    
    print(f'{"Level":<10} {"Steps":<8} {"DIRECT":<10} {"USE":<10} {"CIF":<10} {"Follow%":<10}')
    print('-'*58)
    
    for level in COMPLEXITY_LEVELS:
        a = analysis.get(level, {})
        if 'error' in a or a.get('n', 0) == 0:
            print(f'{level:<10} No data')
            continue
        print(f'{level:<10} {a["avg_steps"]:>5.1f}   {a["accuracy"]["DIRECT"]:>7.1%}   '
              f'{a["accuracy"]["USE"]:>7.1%}   {a["cif_rate"]:>7.1%}   '
              f'{a["followed_wrong_in_cif"]:>7.1%}')

save_json(all_analyses, f'{SAVE_DIR_EXP}/results/analysis_by_complexity.json')

In [ ]:
# ============================================================
# CELL 12: STATISTICAL ANALYSIS
# ============================================================

print('\n' + '='*60)
print('STATISTICAL ANALYSIS: COMPLEXITY → CIF')
print('='*60)

correlation_results = {}

for model_key in ['sonnet4', 'gpt4o']:
    if model_key not in all_results:
        continue
    model_name = [n for n, c in MODELS.items() if c['short'] == model_key][0]
    print(f'\n{model_name}')
    print('-'*50)
    
    # Collect step counts and CIF outcomes per problem
    steps_list = []
    cif_list = []
    
    for level in COMPLEXITY_LEVELS:
        for p in all_results[model_key]['problems'].get(level, []):
            if p['responses']['DIRECT']['correct']:
                steps_list.append(p['steps'])
                cif_occurred = 1 if not p['responses']['USE']['correct'] else 0
                cif_list.append(cif_occurred)
    
    if len(steps_list) > 10 and len(set(cif_list)) > 1:
        r, p_value = stats.pointbiserialr(cif_list, steps_list)
        
        print(f'  N (direct correct): {len(steps_list)}')
        print(f'  Correlation (steps vs CIF): r = {r:.3f}')
        print(f'  p-value: {p_value:.4f}')
        print(f'  Significant: {"Yes" if p_value < 0.05 else "No"}')
        
        correlation_results[model_key] = {
            'n': len(steps_list),
            'correlation': r,
            'p_value': p_value,
            'significant': p_value < 0.05
        }
    else:
        print('  Insufficient data for correlation analysis')
        correlation_results[model_key] = {'error': 'Insufficient data'}

# Trend analysis
print('\n' + '='*60)
print('CIF RATE TREND BY COMPLEXITY')
print('='*60)

for model_key in ['sonnet4', 'gpt4o']:
    if model_key not in all_analyses:
        continue
    analysis = all_analyses[model_key]
    cif_rates = []
    for level in COMPLEXITY_LEVELS:
        a = analysis.get(level, {})
        if 'cif_rate' in a:
            cif_rates.append(a['cif_rate'])
    
    if len(cif_rates) == 3:
        trend = 'increasing' if cif_rates[2] > cif_rates[0] else 'decreasing'
        delta = cif_rates[2] - cif_rates[0]
        print(f'{model_key}: {cif_rates[0]:.1%} → {cif_rates[1]:.1%} → {cif_rates[2]:.1%} ({trend}, Δ={delta:+.1%})')

In [ ]:
# ============================================================
# CELL 13: VISUALIZATION
# ============================================================
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

colors = {'sonnet4': '#5B8FF9', 'gpt4o': '#5AD8A6'}
model_labels = {'sonnet4': 'Claude 4 Sonnet', 'gpt4o': 'GPT-4o'}

# Plot 1: CIF Rate by Complexity
ax1 = axes[0]
x = np.arange(len(COMPLEXITY_LEVELS))
width = 0.35

for i, model_key in enumerate(['sonnet4', 'gpt4o']):
    if model_key not in all_analyses:
        continue
    cif_rates = [all_analyses[model_key].get(level, {}).get('cif_rate', 0) 
                 for level in COMPLEXITY_LEVELS]
    ax1.bar(x + i*width, cif_rates, width, 
            label=model_labels[model_key], color=colors[model_key])

ax1.set_ylabel('CIF Rate', fontsize=12)
ax1.set_title('CIF Rate by Task Complexity', fontsize=14)
ax1.set_xticks(x + width/2)
ax1.set_xticklabels(['Simple\n(1-2 steps)', 'Medium\n(3-4 steps)', 'Complex\n(5+ steps)'])
ax1.legend()
ax1.set_ylim(0, 1)

# Plot 2: Accuracy comparison
ax2 = axes[1]

for i, model_key in enumerate(['sonnet4', 'gpt4o']):
    if model_key not in all_analyses:
        continue
    direct_acc = [all_analyses[model_key].get(level, {}).get('accuracy', {}).get('DIRECT', 0) 
                  for level in COMPLEXITY_LEVELS]
    use_acc = [all_analyses[model_key].get(level, {}).get('accuracy', {}).get('USE', 0) 
               for level in COMPLEXITY_LEVELS]
    
    offset = (i - 0.5) * width
    ax2.bar(x + offset - width/4, direct_acc, width/2, 
            label=f'{model_labels[model_key]} DIRECT', 
            color=colors[model_key], alpha=0.5)
    ax2.bar(x + offset + width/4, use_acc, width/2,
            label=f'{model_labels[model_key]} USE', 
            color=colors[model_key], alpha=1.0)

ax2.set_ylabel('Accuracy', fontsize=12)
ax2.set_title('Accuracy by Complexity & Condition', fontsize=14)
ax2.set_xticks(x)
ax2.set_xticklabels(['Simple', 'Medium', 'Complex'])
ax2.legend(fontsize=8)
ax2.set_ylim(0, 1)

# Plot 3: Follow-wrong rate in CIF cases
ax3 = axes[2]

for i, model_key in enumerate(['sonnet4', 'gpt4o']):
    if model_key not in all_analyses:
        continue
    follow_rates = [all_analyses[model_key].get(level, {}).get('followed_wrong_in_cif', 0) 
                    for level in COMPLEXITY_LEVELS]
    ax3.bar(x + i*width, follow_rates, width,
            label=model_labels[model_key], color=colors[model_key])

ax3.set_ylabel('Follow-Wrong Rate (in CIF)', fontsize=12)
ax3.set_title('How Often CIF Follows Trace Answer', fontsize=14)
ax3.set_xticks(x + width/2)
ax3.set_xticklabels(['Simple', 'Medium', 'Complex'])
ax3.legend()
ax3.set_ylim(0, 1)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR_EXP}/exp_A1_E2_complexity.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n✓ Figure saved')

In [ ]:
# ============================================================
# CELL 14: FINAL SUMMARY
# ============================================================

summary = {
    'experiment_id': 'A1_E2',
    'experiment_name': 'Task Complexity Gradient',
    'date': EXPERIMENT_DATE,
    'hypothesis': 'Complex tasks (more steps) show higher CIF than simple tasks',
    'design': {
        'simple': '1-2 reasoning steps',
        'medium': '3-4 reasoning steps',
        'complex': '5+ reasoning steps'
    },
    'n_problems_per_level': N_PROBLEMS_PER_LEVEL,
    'models': list(MODELS.keys()),
    'results': all_analyses,
    'correlation_analysis': correlation_results,
    'key_findings': []
}

for model_key in ['sonnet4', 'gpt4o']:
    if model_key not in all_analyses:
        continue
    analysis = all_analyses[model_key]
    cif_simple = analysis.get('simple', {}).get('cif_rate', 0)
    cif_complex = analysis.get('complex', {}).get('cif_rate', 0)
    
    summary['key_findings'].append({
        'model': model_key,
        'cif_simple': cif_simple,
        'cif_complex': cif_complex,
        'cif_increase': cif_complex - cif_simple,
        'supports_hypothesis': cif_complex > cif_simple + 0.05,
        'correlation': correlation_results.get(model_key, {})
    })

save_json(summary, f'{SAVE_DIR_EXP}/results/exp_A1_E2_summary.json')

print('\n' + '='*60)
print('EXPERIMENT A1-E2 COMPLETE')
print('='*60)
print(f'\nResults saved to: {SAVE_DIR_EXP}')
print('\n' + '='*60)
print('KEY FINDINGS')
print('='*60)

for finding in summary['key_findings']:
    model_name = [n for n, c in MODELS.items() if c['short'] == finding['model']][0]
    print(f"\n{model_name}:")
    print(f"  Simple CIF:  {finding['cif_simple']:.1%}")
    print(f"  Complex CIF: {finding['cif_complex']:.1%}")
    print(f"  Δ CIF: {finding['cif_increase']:+.1%}")
    print(f"  Supports hypothesis: {'✓ YES' if finding['supports_hypothesis'] else '? No'}")
    corr = finding.get('correlation', {})
    if 'correlation' in corr:
        print(f"  Correlation: r={corr['correlation']:.3f}, p={corr['p_value']:.4f}")

print('\n' + '='*60)